## 텐서플로의 그레이디언트 테이프
텐서플로의 강력한 자동 미분 기능을 활용할 수 있는 API는 GradientTape 이다. 이 API는 파이썬의 with문과 함께 사용하여 해당 코드 블록 안의 모든 텐서 연산을 계산 그래프 형태(이따금 이를 '테이프(tape)' 라고 부릅니다)로 기록한다. 그 다음 이 그래프를 사용하여 (tf.Variable 클래스의 인스턴스인) 변수 또는 변수 집합에 대한 어떤 추력의 그레이디언트도 계산할 수 있다. <br><br>
`tf.Variable`은 변경가능한 상태를 담기위한 특별한 종류의 텐서이다. 예를 들어 신경망의 가중치는 항상 `tf.Variable`의 인스턴스

In [111]:
import tensorflow as tf
# 스칼라 변수
x = tf.Variable(0.)
x

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.0>

In [112]:
x = tf.Variable([0.])
x

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>

In [113]:
# GradientTape 블록 시작
with tf.GradientTape() as tape :
  y = 2*x + 3

# 이후 tape를 사용해 변수 x에 대한 출력 y의 그레이디언트 계산하보기
grad_of_y_wrt_x = tape.gradient(y, x)
grad_of_y_wrt_x

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>

In [114]:
# 다차원 텐서와 함께 사용할 수도 있다.
x = tf.Variable(tf.zeros((2,2)))

with tf.GradientTape() as tape :
  y = 2*x + 3

# grad_of_y_wrt_x는 (x와 크기가 같은)(2,2) 크기의 텐서로
# x=[[0,0], [0,0]]일때 y=2*x + 3의 곡률을 나타낸다.
grad_of_y_wrt_x = tape.gradient(y, x)
grad_of_y_wrt_x

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

리스트 형태의 그래이디언트를 계산할 수도 있다. W의 지점에서 미분한 값을 보는거 같은데 일단 예제를 따라해보겠다

In [115]:
W = tf.Variable(tf.random.uniform((2,2)))
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2,2))

with tf.GradientTape() as tape :
  # matmul은 텐서플로의 점곱함수
  y = tf.matmul(x, W), b

# grad_of_y_wrt_W_and_b는 2개의 텐서를 담은 리스트
# 각 텐서는 W, b와 크기가 같다
grad_of_y_wrt_W_and_b = tape.gradient(y, [W, b])
grad_of_y_wrt_W_and_b

[<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1.3151661, 1.3151661],
        [0.8426603, 0.8426603]], dtype=float32)>,
 <tf.Tensor: shape=(2,), dtype=float32, numpy=array([1., 1.], dtype=float32)>]

## 텐서플로를 사용하여 첫번째 예제를 밑바닥부터 구현해보기
케라스 기능을 사용하지 않은 저수준의 구현으로 첫 번째 예제를 다시 살펴보겠다. 구체적인 구현을 통해 딥러닝 이면의 수학 이론을 명확히게 이해하도록 돕는 것이 목적이니 진행되는 내용의 핵심을 파악해볼 수 있도록 해보자.

In [116]:
# 단순한 Dense 클래스
import tensorflow as tf

class NaiveDense :
  # activation : 각 원소에 적용되는 함수
  def __init__(self, input_size, output_size, activation) :
    self.activation = activation

    # w와 b는 모델 파라미터 변수
    # 랜덤한 값으로 초기화된 (input_size, output_size) 크기의 행렬 W 생성
    w_shape = (input_size, output_size)
    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
    self.W = tf.Variable(w_initial_value)

    # 0으로 초기화된 (output_size,) 크기의 벡터 b를 만든다
    b_shape = (output_size,)
    b_initial_value = tf.zeros(b_shape)
    self.b = tf.Variable(b_initial_value)


  # 정방향 패스 수행
  def __call__(self, inputs) :
    # W는 행렬이고 b는 벡터
    # 행렬을 입력으로 받고, 그 입력 텐서의 새로운 표현인 또 다른 행렬을 반환하는 함수
    # 입력텐서와 텐서W 사이에 matmul을 해주어 새로운 행렬 만들어주기
    # 그 행렬과 벡터 b 를 덧셈해주고
    # 그 값을 activation 연산 해준다.
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  # 층의 가중치를 추출하기 위한 메서드
  def weights(self) :
    return [self.W, self.b]

## @ (decorator)
[ Java ]<br>
annotation; 어노테이션, 애너테이션;<br>
[ Python ]<br>
decorator; 데코레이터, 장식자;<br><br>
함수(메서드)를 장식한다고 하여 이런 이름이 붙었다. 함수를 수정하지 않은 상태에서 추가 기능을 구현할 때 사용한다.
```python
@ 데코레이터
def 함수이름() :
  코드
```
위처럼 호출할 함수위에 @ 데코레이터 형식으로 지정하여 사용할 수 있다.
```python
def trace(func):                             # 호출할 함수를 매개변수로 받음
    def wrapper():                           # 호출할 함수를 감싸는 함수
        print(func.__name__, '함수 시작')    # __name__으로 함수 이름 출력
        func()                               # 매개변수로 받은 함수를 호출
        print(func.__name__, '함수 끝')
    return wrapper                           # wrapper 함수 반환

def hello():
    print('hello')

def world():
    print('world')

trace_hello = trace(hello)    # 데코레이터에 호출할 함수를 넣음
trace_hello()                 # 반환된 함수를 호출
trace_world = trace(world)    # 데코레이터에 호출할 함수를 넣음
trace_world()                 # 반환된 함수를 호출
```
원래라면 위와 같이 작성 후 사용했겠지만 데코레이터를 사용한다면 호출부분이 더욱 간단하게 변한다
```python
def trace(func):                             # 호출할 함수를 매개변수로 받음
    def wrapper():
        print(func.__name__, '함수 시작')    # __name__으로 함수 이름 출력
        func()                               # 매개변수로 받은 함수를 호출
        print(func.__name__, '함수 끝')
    return wrapper                           # wrapper 함수 반환

@trace    # @데코레이터
def hello():
    print('hello')

@trace    # @데코레이터
def world():
    print('world')

hello()    # 함수를 그대로 호출
world()    # 함수를 그대로 호출
```
이렇게 hellow와 worl 함수 위에 @trace를 붙여 작성한 후 hello와 world 함수를 그대로 호출하는 것이다.
<br><br>
함수에는 데코레이터를 여러개 지정할 수도 있다. 함수위에 여러줄로 지정해주면 되는데 이때 데코레이터가 실행되는 순서는 위에서 아래 순이다
```python
def decorator1(func):
    def wrapper():
        print('decorator1')
        func()
    return wrapper

def decorator2(func):
    def wrapper():
        print('decorator2')
        func()
    return wrapper

# 데코레이터를 여러 개 지정
@decorator1
@decorator2
def hello():
    print('hello')

hello()
```
```
decorator1
decorator2
hello
```

## `__call__()`
```python
class Test :
  def __call__(self, num) :
    if num == 1 :
      print('Hello')
    else :
      # 에러 발생시키기
      raise

test = Test()
test()(1)
>> Hello
test(1)
>> Hello
test(2)
>> error
```
- `__init__()`은 생성자이며 인스턴스 초기화를 위해 작성
- `__call__()`은 인스턴스가 호출되었을 때 실행되는 메서드

예를 들어 간단한 클래스를 생성해보자면
```python
class Hap :
  def __init__(self, n1, n2) :
    self.n1 = n1
    self.n2 = n2
    return print(self.n1, self.n2)

  def __call__(self, n1, n2) :
    self.n1 = n1
    self.n2 = n2
    return print(self.n1 + self.n2)

h = Hap(1, 2)
>> 1 2 >> init
h(7, 8)
>> 15  >> call
```
이를 보면 init으로 클래스를 생성하고 call로 정의된 것을 함수처럼 불러온다는 것을 확인해볼 수 있었다.

In [117]:
# 단순한 Sequential 클래스
# 층의 리스트를 받고 __call__() 메서드에서 입력을 사용하여 층을 순서대로 호출
class NaiveSequential :
  def __init__(self, layers) :
    self.layers = layers

  def __call__(self, inputs) :
    x = inputs
    for layer in self.layers :
      x = layer(x)
    return x

  # 층의 파라미터를 쉽게 구할 수 있도록 weights 속성 제공
  @property
  def weights(self) :
    weights = []
    for layer in self.layers :
      weights += layer.weights
    return weights

In [118]:
# 이제 NaiveDense 클래스와 NaiveSequential 클래스를 사용하여
# 케라스와 유사한 모델을 만들어볼 수 있다.
model = NaiveSequential([
    NaiveDense(input_size=28*28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

# 만약 model.weights의 길이가 4가 아니라면 에러를 발생시켜라
# 모델이 제대로 만들어졌는지 확인
assert len(model.weights) == 4

## assert
뒤의 조건이 True가 아니면 AssertError를 발생시킨다. 어떠한 함수가 반드시 정수만 받도록 하고 싶다면, 이 함수에 들어오는 것이 정수인지 확인할 필요가 있을 것이다. 이를 위해 if문을 사용할 수도 있고 예외처리를 사용할수도 있겠지만 assert(가정 설정문)을 사용하는 방법도 있는 것이다.
```python
assert 조건, '메시지'
```
위의 형태로 작성할 수 있으며, 이때 '메시지'는 생략할 수도 있다.

In [119]:
# 배치 제너레이터
import math

class BatchGenerator :
  def __init__(self, images, labels, batch_size = 128) :
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images)/batch_size)

  def next(self) :
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

### 훈련 스텝 실행하기
한 배치 데이터에서 모델을 실행하고 가중치를 업데이트 하는 일인 '훈련스텝'을 구현하기 위해서는 다음을 유의해봐야 한다.

1. 배치에 있는 이미지에 대하여 모델의 예측을 계산
2. 실제 레이블을 사용하여 이 예측의 손실 값을 계산
3. 모델 가중치에 대한 손실의 그레이디언트를 계산
4. 이 그레이디언트의 반대 방향으로 가중치를 조금 이동

이제 이 그레이디언트를 계산하기 위해 앞서 봤던 GradientTape 객체를 사용해보자

In [120]:
def one_training_step(model, images_batch, labels_batch) :
  # 정방향 패스를 실행
  # GradientTape 블록 안에서 모델의 예측을 계산하게 된다
  with tf.GradientTape() as tape :
    predictions = model(images_batch)
    per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
    average_loss = tf.reduce_mean(per_sample_losses)

  # 가중치에 대한 손실 그레이디언트를 계산하는 부분이다
  # gradients 리스트의 각 항목은 model.weights 리스트에 있는 가중치에 매칭
  gradients = tape.gradient(average_loss, model.weights)

  # 이 그레이디언트를 사용하여 가중치를 업데이트하는 부분이다
  update_weights(gradients, model.weights)
  return average_loss

In [121]:
# 가중치를 업데이트 하는 함수(케라스의 Optimizer 인스턴스기능을 수동 구현 중)
# 목적은 이 배치의 손실을 감소시키기 위한 방향으로 가중치를 '조금' 이동하는 것이다.
# 이동의 크기는 '학습률'에 의해 결정되며 일반적으로 작은 값
learning_rate = 1e-3

# update_weights 함수를 구현하는 가장 간단한 방법은 각 가중치에서 (gradient*learning_rate)를 빼는 것이다.
def update_weights(gradients, weights) :
  for g, w in zip(gradients, weights) :
    # 텐서플로의 assign_sub 메서드는 -=와 동일
    w.assign_sub(g * learning_rate)

In [122]:
# train의 epochs하나는 단순히 train data의 각 batch에 대한 train step을 반복하는 것이다
# 따라서 전체 train loop는 단순히 이 에폭의 반복이라고 생각해볼 수 있다.
def fit(model, images, labels, epochs, batch_size=128) :
  for epoch_counter in range(epochs) :
    print(f'에폭 {epoch_counter}')
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches) :
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0 :
        print(f'{batch_counter}번째 배치 손실: {loss:.2f}')

In [123]:
# 이제 mnist를 가져와 함수들을 테스트 해보자
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000, 28*28))
test_images = test_images.astype('float32')/255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에폭 0
0번째 배치 손실: 8.79
100번째 배치 손실: 2.23
200번째 배치 손실: 2.19
300번째 배치 손실: 2.09
400번째 배치 손실: 2.23
에폭 1
0번째 배치 손실: 1.90
100번째 배치 손실: 1.86
200번째 배치 손실: 1.81
300번째 배치 손실: 1.71
400번째 배치 손실: 1.83
에폭 2
0번째 배치 손실: 1.57
100번째 배치 손실: 1.57
200번째 배치 손실: 1.49
300번째 배치 손실: 1.42
400번째 배치 손실: 1.51
에폭 3
0번째 배치 손실: 1.31
100번째 배치 손실: 1.33
200번째 배치 손실: 1.22
300번째 배치 손실: 1.20
400번째 배치 손실: 1.27
에폭 4
0번째 배치 손실: 1.11
100번째 배치 손실: 1.15
200번째 배치 손실: 1.03
300번째 배치 손실: 1.04
400번째 배치 손실: 1.10
에폭 5
0번째 배치 손실: 0.97
100번째 배치 손실: 1.01
200번째 배치 손실: 0.89
300번째 배치 손실: 0.92
400번째 배치 손실: 0.98
에폭 6
0번째 배치 손실: 0.86
100번째 배치 손실: 0.90
200번째 배치 손실: 0.79
300번째 배치 손실: 0.83
400번째 배치 손실: 0.90
에폭 7
0번째 배치 손실: 0.78
100번째 배치 손실: 0.82
200번째 배치 손실: 0.72
300번째 배치 손실: 0.76
400번째 배치 손실: 0.83
에폭 8
0번째 배치 손실: 0.72
100번째 배치 손실: 0.75
200번째 배치 손실: 0.66
300번째 배치 손실: 0.70
400번째 배치 손실: 0.78
에폭 9
0번째 배치 손실: 0.67
100번째 배치 손실: 0.70
200번째 배치 손실: 0.61
300번째 배치 손실: 0.66
400번째 배치 손실: 0.74


In [124]:
import numpy as np

# 모델 평가하기
pred = model(test_images)
# 텐서플로 텐서의 .numpy() 메서드를 호출하여 넘파이 배열로 바꾼다
pred = pred.numpy()
# 테스트 이미지에 대한 예측에 argmax 함수를 적용
pred_labels = np.argmax(pred, axis=1)
matches = pred_labels == test_labels
print(f'정확도 : {matches.mean() : .2f}')

정확도 :  0.82


## Conv2D()
컨볼루션 신경망(CNN)은 입력된 이미지에서 다시 한번 특징을 추출하기 위해 마스크(필터, 윈도 또는 커널)을 도입하는 기법이다. 이 마스크를 통해 새롭게 만들어진 층을 컨볼루션(합성곱)이라부른다. 이 컨볼루션을 만들면 입력 데이터로부터 더욱 정교한 특징을 추출할 수 있고, 마스크를 여러개 만들 경우 여러개의 컨볼루션이 만들어진다<br><br>
케라스에서 컨볼루션 층을 추가하는 함수는 Conv2D()이며 아래와 같이 컨볼루션 층을 이용해볼 수 있다.
```python
Conv2D(32, kernel_size=(3,3), input_shape=(28, 28, 1), activation='relu')
```
- 첫번째 인자 : 마스크를 몇 개 적용할지 정한다. 여러개의 마스크를 적용하면 서로다른 컨볼루션이 여러 개 나오는데. 여기서는 32개의 마스크를 적용했다. 보통 32, 64, 128 마스크를 사용하며 몇 개의 마스크를 사용할지 기준은 없다고한다. 여러번 다르게 적용시켜보며 가장 잘 맞는 마스크 개수를 찾아야 할 듯 싶다.
- Kernel_size : 마스크(커널)의 크기를 정한다. kernel_size=(행, 열) 형식으로 정하며, 여기서는 3x3 크기의 마스크를 사용하게끔 정했다. 그냥 `kernel_size=3`으로 적어주어도 된다.
- input_shape : Dense 층과 마찬가지로 맨 처음 층에는 입력되는 값을 알려주어야 한다. input_shape=(행, 열, 색상 또는 흑백) 형식으로 정한다
- activation : 활성화 함수를 정의한다.